In [6]:
import pandas as pd
import numpy as np
import config
from sqlalchemy import text
print(config.version)
print(config.connection_string)
pd.set_option('display.max_columns', None)
pd.options.mode.copy_on_write = True

3.2.0.Dev
postgresql://postgres:sa@localhost/icip


In [7]:
specialite=21
year=23
dept=6
source="PA"
time=30
time_type="HC"
# accessibilite_exp=-0.12 #0.08 pour 45
accessibilite_exp=-(75-time)*4/1500
print(year, dept, specialite, source, time, time_type, accessibilite_exp)

23 6 21 PA 30 HC -0.12


In [8]:
def get_ps(year, specialite):
    sql=f"""
select ps.id, ps.nom, ps.prenom, an.dept_id, an.id adresse_norm_id, i.id iris, an.lon, an.lat, 'L' as code_mode_exercice from ps
join tarif t on t.ps_id = ps.id
join tarif_date_source tds on tds.tarif_id=t.id
join cabinet c on t.cabinet_id=c.id
join adresse_raw ar on c.adresse_raw_id=ar.id
join adresse_norm an on ar.adresse_norm_id=an.id
join specialite_profession sp on sp.profession_id=t.profession_id
join specialite s on s.id=sp.specialite_id
join iris.iris i on i.code=an.iris
where sp.specialite_id={specialite}
and tds.date_source_id >= {year}00 and  tds.date_source_id < {year+1}00
and ar.dept_id={dept}
group by ps.id, c.id, an.id, i.id
"""
    print(f"Quering PS for year {year} and specialite {specialite}")
    return pd.read_sql(sql, config.connection_string)

In [12]:
def get_pa(year, specialite, is_medecin=True):
    if is_medecin:
        sql = f"""
select pa.id, pa.nom, pa.prenom, an.dept_id, an.id adresse_norm_id, i.id iris, an.lon, an.lat, ps.code_mode_exercice
from apl.ps_libreacces ps
join personne_activite pa on pa.inpp=ps.inpp
join pa_adresse_norm_date_source pands on pands.personne_activite_id=pa.id and pands.date_source_id=ps.date_source_id
join personne_activite_diplome pad on pad.personne_activite_id=pa.id
join diplome d on d.code_diplome=ps.code_diplome
join profession_diplome pd on pd.diplome_id=d.id
join profession p on p.id=pd.profession_id
join specialite_profession sp on sp.profession_id=p.id
join adresse_norm an on an.id=pands.adresse_norm_id
join date_source ds on ds.id=pands.date_source_id
join iris.iris i on i.code=an.iris
where sp.specialite_id={specialite}
and ps.code_mode_exercice='L'
and ds.annee={year}
and pands.adresse_norm_id is not null
and an.dept_id={dept}
group by pa.id, an.id, i.id, ps.code_mode_exercice
"""
    else:
        sql = f"""
select pa.id, pa.nom, pa.prenom, an.dept_id, an.id adresse_norm_id, i.id iris, an.lon, an.lat, ps.code_mode_exercice
from apl.ps_libreacces ps
join personne_activite pa on pa.inpp=ps.inpp
join pa_adresse_norm_date_source pands on pands.personne_activite_id=pa.id and pands.date_source_id=ps.date_source_id
join profession_code_profession pcp on pcp.code_profession_id=ps.code_profession
join specialite_profession sp on sp.profession_id=pcp.profession_id
join adresse_norm an on an.id=pands.adresse_norm_id
join date_source ds on ds.id=pands.date_source_id
join iris.iris i on i.code=an.iris
where sp.specialite_id={specialite}
and ps.code_mode_exercice='L'
and ds.annee={year}
and pands.adresse_norm_id is not null
and an.dept_id={dept}
group by pa.id, an.id, i.id, ps.code_mode_exercice
"""
    print(f"Quering PA for year {year} and specialite {specialite}")
    print(sql)
    return pd.read_sql(sql, config.connection_string)


In [13]:
def get_by_source(year, specialite, source):
    if source=="PS":
        return get_ps(year, specialite)
    elif source=="PA":
        return get_pa(year, specialite, specialite<=20)
    else:
        raise ValueError(f"Bad source: {source}")
    

In [15]:
df = get_by_source(year, specialite, source)
df

Quering PA for year 23 and specialite 21

select pa.id, pa.nom, pa.prenom, an.dept_id, an.id adresse_norm_id, i.id iris, an.lon, an.lat, ps.code_mode_exercice
from apl.ps_libreacces ps
join personne_activite pa on pa.inpp=ps.inpp
join pa_adresse_norm_date_source pands on pands.personne_activite_id=pa.id and pands.date_source_id=ps.date_source_id
join profession_code_profession pcp on pcp.code_profession_id=ps.code_profession
join specialite_profession sp on sp.profession_id=pcp.profession_id
join adresse_norm an on an.id=pands.adresse_norm_id
join date_source ds on ds.id=pands.date_source_id
join iris.iris i on i.code=an.iris
where sp.specialite_id=21
and ps.code_mode_exercice='L'
and ds.annee=23
and pands.adresse_norm_id is not null
and an.dept_id=6
group by pa.id, an.id, i.id, ps.code_mode_exercice



,id,nom,prenom,dept_id,adresse_norm_id,iris,lon,lat,code_mode_exercice
0,5050,BENAROUS,FRANCOISE,6,1213487,60110000,7.326919,43.702660,L
1,5051,BUTTAFOGHI,PASCALE,6,1216641,60882504,7.253609,43.699982,L
2,5062,BUSSI,SYLVIA,6,461218,60330103,7.190962,43.774561,L
3,5170,DRAI,BRIGITTE,6,1161690,60881302,7.253018,43.718900,L
4,5201,PUSCH,AUDREY,6,2519,60880304,7.286355,43.704275,L
...,...,...,...,...,...,...,...,...,...
3133,2524286,CHAPELON,ROXANNE,6,1179636,61130000,7.468433,43.789429,L
3134,2527313,ALBIN,FABRICE,6,749200,60881501,7.265188,43.707962,L
3135,2529192,ROSATI,ELODIE,6,1179636,61130000,7.468433,43.789429,L
3136,2529835,LAPOUBLE,AMBRE,6,741482,60300109,7.006474,43.569567,L


In [16]:
nb_ps = df["id"].nunique()
unique_ps = df.groupby(["id", "lon", "lat"])
print(f"Nb PS {nb_ps}")
print(f"Nb unique PS {len(unique_ps)}")

Nb PS 2775
Nb unique PS 3138


In [17]:
df["key"] = df["id"].astype(str) + "_" + df["lat"].astype(str) + "_" + df["lon"].astype(str)
df["nb_cabinet"] = df.groupby(["id"])["key"].transform("nunique")
df["nb_cabinet"].value_counts(normalize=True)

nb_cabinet
1    0.771511
2    0.220523
3    0.006692
4    0.001275
Name: proportion, dtype: float64

In [18]:
df["weight"] = 1 / df["nb_cabinet"]
df = df[df["code_mode_exercice"]=="L"]
df["nb"] = df.groupby("iris")["weight"].transform("sum")
df.head(10)

,id,nom,prenom,dept_id,adresse_norm_id,iris,lon,lat,code_mode_exercice,key,nb_cabinet,weight,nb
0,5050,BENAROUS,FRANCOISE,6,1213487,60110000,7.326919,43.702660,L,5050_43.70266_7.326919,1,1.0,13.500000
1,5051,BUTTAFOGHI,PASCALE,6,1216641,60882504,7.253609,43.699982,L,5051_43.699982_7.253609,1,1.0,3.000000
2,5062,BUSSI,SYLVIA,6,461218,60330103,7.190962,43.774561,L,5062_43.774561_7.190962,1,1.0,7.666667
3,5170,DRAI,BRIGITTE,6,1161690,60881302,7.253018,43.718900,L,5170_43.7189_7.253018,1,1.0,24.500000
4,5201,PUSCH,AUDREY,6,2519,60880304,7.286355,43.704275,L,5201_43.704275_7.286355,1,1.0,52.000000
5,5211,CHALE,MURIELLE,6,81980,60040101,7.091032,43.600009,L,5211_43.600009_7.091032,1,1.0,4.000000
6,5273,DANDALEIX,THIERRY,6,1220356,60290112,7.016715,43.558273,L,5273_43.558273_7.016715,1,1.0,37.000000
7,5287,DOMINGUES,AUDREY,6,293376,61520103,7.039946,43.621395,L,5287_43.621395_7.039946,1,1.0,11.500000
8,6396,RADINEZ,KHADIJA,6,993601,61050102,7.047759,43.665797,L,6396_43.665797_7.047759,1,1.0,9.000000
9,6689,DERRIEN,VIRGINIE,6,389864,60690107,6.949348,43.639920,L,6689_43.63992_6.949348,1,1.0,24.000000


In [19]:
df = df.sort_values(by='iris')
df = df.drop_duplicates(subset=['iris', 'nb'])
df


,id,nom,prenom,dept_id,adresse_norm_id,iris,lon,lat,code_mode_exercice,key,nb_cabinet,weight,nb
202,95591,LESUEUR,AURELIE,6,81980,60040101,7.091032,43.600009,L,95591_43.600009_7.091032,1,1.0,4.00
2074,985120,BEN YOUSSEF,HASSINE,6,1309164,60040102,7.068795,43.597024,L,985120_43.597024_7.068795,1,1.0,2.00
2006,943164,MERAU,ALEXIS,6,502784,60040103,7.127899,43.556312,L,943164_43.556312_7.127899,1,1.0,2.00
1467,679482,BONHOMME,JOEL,6,1213521,60040105,7.105166,43.600671,L,679482_43.600671_7.105166,1,1.0,1.00
2030,960643,GUENIN,LYDIA,6,2007,60040106,7.112369,43.606074,L,960643_43.606074345716316_7.112369457663655,2,0.5,5.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...
694,327149,GARCIA,CORALIE,6,1190804,61610105,7.133696,43.635700,L,327149_43.6357_7.133696,2,0.5,5.25
2306,1110521,VILLAUME,SYLVIE,6,444626,61610106,7.117664,43.635814,L,1110521_43.635814_7.117664,1,1.0,6.00
1817,854519,LICARI,STEPHANIE,6,398477,61610107,7.124276,43.655477,L,854519_43.655477_7.124276,1,1.0,5.00
1514,710022,LAURENT,CAROLINE,6,392975,61630000,7.593599,44.087694,L,710022_44.087694_7.593599,1,1.0,3.00


In [20]:
def get_iris_matrix(time_type: str):
    sql = f"""
(select iris.id "iris1", iris.id "iris2", 0 "km", 0 "time_hc", 0 "time_hp" from iris.iris
where id / 10000000 = {dept}
union
select iris_id_from as "iris1", iris_id_to "iris2", route_km "km", route_min "time_hc", route_hp_min "time_hp" from iris.iris_matrix
where (iris_id_from/10000000={dept} or iris_id_to/10000000={dept})
and route_min <= {time}) order by "iris1", "iris2"
"""
    print(sql)
    m = pd.read_sql(sql, config.connection_string)
    m["time"] = m[f"time_{time_type.lower()}"].copy()
    return m

iris_matrix = get_iris_matrix(time_type)
# iris_matrix = pd.read_csv("od_06_sql_rename.csv")
iris_matrix["iris"] = iris_matrix["iris2"].astype("int64")
iris_matrix
    


(select iris.id "iris1", iris.id "iris2", 0 "km", 0 "time_hc", 0 "time_hp" from iris.iris
where id / 10000000 = 6
union
select iris_id_from as "iris1", iris_id_to "iris2", route_km "km", route_min "time_hc", route_hp_min "time_hp" from iris.iris_matrix
where (iris_id_from/10000000=6 or iris_id_to/10000000=6)
and route_min <= 30) order by "iris1", "iris2"



,iris1,iris2,km,time_hc,time_hp,time,iris
0,40080000,60530000,17,22,22,22,60530000
1,40080000,60990000,21,24,24,24,60990000
2,40390000,61540000,24,30,30,30,61540000
3,40420000,60530000,11,19,19,19,60530000
4,40420000,60710000,22,30,32,30,60710000
...,...,...,...,...,...,...,...
117764,831380000,60850103,34,29,35,29,60850103
117765,831380000,60950101,18,19,21,19,60950101
117766,831380000,60950102,19,22,25,22,60950102
117767,831380000,60950103,18,20,23,20,60950103


In [21]:
def get_pop_iris():
    yy=min(21, year)
    sql = f"""
select i.id iris, pi.iris iris_string, c.code code_commune, i.type type_iris, pi.pop, pi.pop0002, pi.pop0305, pi.pop0610, pi.pop1117, pi.pop1824, pi.pop2539, pi.pop4054, pi.pop5564, pi.pop6579, pi.pop80p
from iris.pop_iris pi
join iris.iris i on  pi.iris=i.code
join iris.commune c on i.commune_id=c.id
where com like '{dept:02}%'
and year={yy}
"""
    print(sql)
    return pd.read_sql(text(sql), config.connection_string)

pop_iris = get_pop_iris()
pop_iris


select i.id iris, pi.iris iris_string, c.code code_commune, i.type type_iris, pi.pop, pi.pop0002, pi.pop0305, pi.pop0610, pi.pop1117, pi.pop1824, pi.pop2539, pi.pop4054, pi.pop5564, pi.pop6579, pi.pop80p
from iris.pop_iris pi
join iris.iris i on  pi.iris=i.code
join iris.commune c on i.commune_id=c.id
where com like '06%'
and year=21



,iris,iris_string,code_commune,type_iris,pop,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4054,pop5564,pop6579,pop80p
0,60120105,060120105,06012,A,3136.830308,68.356436,55.185368,179.895476,266.832339,226.432893,571.248108,831.040803,391.639115,351.412985,194.786785
1,60500000,060500000,06050,C,526.000000,14.188627,14.119226,40.517897,47.603916,17.493201,71.924387,145.187085,83.001180,65.334467,26.630014
2,60720000,060720000,06072,C,142.000000,0.000000,6.761905,4.829932,4.829932,3.863946,21.251701,27.047619,27.047619,31.877551,14.489796
3,61050101,061050101,06105,A,4007.649525,96.881691,145.234138,320.060918,454.509675,180.143151,475.809676,953.212078,561.152910,592.431015,228.214272
4,60150000,060150000,06015,C,1227.000000,35.622418,38.520557,52.247192,81.918133,62.907751,196.126186,266.670404,226.738245,194.355336,71.893777
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,60883403,060883403,06088,A,1708.177920,61.674796,46.078830,82.091539,91.537698,149.477993,262.631990,280.939824,240.042775,297.894294,195.808181
496,60883501,060883501,06088,A,2649.665320,62.951120,77.186422,127.397604,250.535242,185.758643,326.558748,654.569566,408.285147,404.624607,151.798222
497,60883601,060883601,06088,A,2930.552478,97.864784,111.681073,140.331825,237.887267,182.415424,467.256394,527.349301,425.050263,564.881081,175.835066
498,60883602,060883602,06088,A,2200.226082,66.010794,74.394021,148.464982,170.842823,114.243600,378.151835,433.302933,292.715762,441.808746,80.290586


In [22]:
apl = iris_matrix.merge(pop_iris, on="iris", how="left", suffixes=('', ''))
apl

,iris1,iris2,km,time_hc,time_hp,time,iris,iris_string,code_commune,type_iris,pop,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4054,pop5564,pop6579,pop80p
0,40080000,60530000,17,22,22,22,60530000,060530000,06053,C,143.000000,3.042553,3.042553,3.042553,15.212766,3.042553,19.269504,22.312057,20.283688,39.553191,14.198582
1,40080000,60990000,21,24,24,24,60990000,060990000,06099,C,1813.000000,44.960846,59.174287,115.477201,170.664745,99.908923,286.028889,372.666297,213.784254,309.368287,140.966272
2,40390000,61540000,24,30,30,30,61540000,061540000,06154,C,495.000000,13.220641,11.542128,38.449393,35.331972,16.699780,71.015373,85.106947,76.715979,113.173907,33.743879
3,40420000,60530000,11,19,19,19,60530000,060530000,06053,C,143.000000,3.042553,3.042553,3.042553,15.212766,3.042553,19.269504,22.312057,20.283688,39.553191,14.198582
4,40420000,60710000,22,30,32,30,60710000,060710000,06071,C,584.000000,12.947819,10.957307,29.872821,24.887047,20.940392,91.782711,104.799327,98.895928,130.928878,57.987769
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117764,831380000,60850103,34,29,35,29,60850103,060850103,06085,A,2603.604864,57.428416,72.207652,135.459522,194.409719,141.793750,318.526897,479.908067,393.484524,562.460591,247.925725
117765,831380000,60950101,18,19,21,19,60950101,060950101,06095,A,3223.464405,77.910796,87.987431,210.729551,255.894450,155.595176,478.030563,685.139541,418.753123,589.352449,264.071324
117766,831380000,60950102,19,22,25,22,60950102,060950102,06095,A,2563.685976,67.975915,77.960010,155.903079,226.090400,145.568264,401.767992,529.572959,354.290116,427.490829,177.066411
117767,831380000,60950103,18,20,23,20,60950103,060950103,06095,A,2468.849619,46.976690,68.925850,125.728002,224.736092,103.186116,259.487677,537.700844,388.050677,528.083677,185.973993


In [23]:
accessibilite_fn = lambda x: np.exp(accessibilite_exp * x)

# apl["time"] = apl["time"].replace(0, 0.5)
apl["accessibilite_weight"] = accessibilite_fn(apl["time"])
apl.head(10)

,iris1,iris2,km,time_hc,time_hp,time,iris,iris_string,code_commune,type_iris,pop,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4054,pop5564,pop6579,pop80p,accessibilite_weight
0,40080000,60530000,17,22,22,22,60530000,060530000,06053,C,143.0,3.042553,3.042553,3.042553,15.212766,3.042553,19.269504,22.312057,20.283688,39.553191,14.198582,0.071361
1,40080000,60990000,21,24,24,24,60990000,060990000,06099,C,1813.0,44.960846,59.174287,115.477201,170.664745,99.908923,286.028889,372.666297,213.784254,309.368287,140.966272,0.056135
2,40390000,61540000,24,30,30,30,61540000,061540000,06154,C,495.0,13.220641,11.542128,38.449393,35.331972,16.699780,71.015373,85.106947,76.715979,113.173907,33.743879,0.027324
3,40420000,60530000,11,19,19,19,60530000,060530000,06053,C,143.0,3.042553,3.042553,3.042553,15.212766,3.042553,19.269504,22.312057,20.283688,39.553191,14.198582,0.102284
4,40420000,60710000,22,30,32,30,60710000,060710000,06071,C,584.0,12.947819,10.957307,29.872821,24.887047,20.940392,91.782711,104.799327,98.895928,130.928878,57.987769,0.027324
5,40420000,60990000,17,24,24,24,60990000,060990000,06099,C,1813.0,44.960846,59.174287,115.477201,170.664745,99.908923,286.028889,372.666297,213.784254,309.368287,140.966272,0.056135
6,40420000,61240000,15,30,30,30,61240000,061240000,06124,C,52.0,2.557377,0.852459,0.000000,0.000000,0.852459,6.819672,4.262295,7.672131,23.016393,5.967213,0.027324
7,40430000,60020000,15,20,20,20,60020000,060020000,06002,C,51.0,0.944444,0.000000,0.944444,0.000000,4.722222,2.833333,12.277778,12.277778,14.166667,2.833333,0.090718
8,40430000,60240000,8,11,11,11,60240000,060240000,06024,C,184.0,4.380952,5.476190,4.380952,16.428571,3.285714,19.714286,36.142857,20.809524,55.857143,17.523810,0.267135
9,40430000,60450000,18,23,23,23,60450000,060450000,06045,C,76.0,2.710965,0.903655,0.903655,1.807310,4.518275,11.840499,5.421930,17.169444,22.591374,8.132895,0.063292


In [24]:
apl[apl["iris1"]==61290000]

,iris1,iris2,km,time_hc,time_hp,time,iris,iris_string,code_commune,type_iris,pop,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4054,pop5564,pop6579,pop80p,accessibilite_weight
105136,61290000,60730000,14,17,17,17,60730000,060730000,06073,C,653.0,15.966553,14.947243,42.905088,55.876926,20.971505,96.981393,145.940458,117.149194,107.190565,35.071074,0.130029
105137,61290000,60800000,10,18,19,18,60800000,060800000,06080,C,105.0,4.952830,1.981132,3.962264,3.962264,2.971698,10.896226,18.820755,16.839623,35.660377,4.952830,0.115325
105138,61290000,60820000,23,30,30,30,60820000,060820000,06082,C,115.0,0.833333,0.833333,2.500000,6.666667,5.833333,13.333333,30.000000,18.333333,25.000000,11.666667,0.027324
105139,61290000,61020000,13,21,21,21,61020000,061020000,06102,C,133.0,5.181818,1.727273,4.318182,10.363636,4.318182,12.954545,21.590909,31.954545,29.363636,11.227273,0.080460
105140,61290000,61100000,12,19,19,19,61100000,061100000,06110,C,116.0,2.924370,2.924370,2.924370,6.823529,4.873950,11.697479,24.369748,35.092437,16.571429,7.798319,0.102284
105141,61290000,61110000,10,17,17,17,61110000,061110000,06111,C,113.0,0.000000,0.000000,1.899160,3.798319,2.848739,15.193277,22.789916,26.588235,25.638655,14.243697,0.130029
105142,61290000,61290000,0,0,0,0,61290000,061290000,06129,C,301.0,1.104023,4.449030,15.804142,40.071305,15.409834,45.894488,60.413865,35.273148,57.768530,24.811635,1.000000
105143,61290000,61460000,18,23,23,23,61460000,061460000,06146,C,149.0,5.000000,3.000000,11.000000,16.000000,7.000000,23.000000,35.000000,28.000000,13.000000,8.000000,0.063292
105144,61290000,61530000,14,21,21,21,61530000,061530000,06153,C,835.0,18.020482,23.026074,44.052137,67.079572,41.031287,120.144366,165.201707,152.189294,156.194759,48.060321,0.080460


In [25]:
cols = [col for col in apl.columns if "pop" in col and col != "pop"]
cols

['pop0002',
 'pop0305',
 'pop0610',
 'pop1117',
 'pop1824',
 'pop2539',
 'pop4054',
 'pop5564',
 'pop6579',
 'pop80p']

In [30]:
def get_over(year, specialite, is_medecin):
    yy=min(year, 24)
    if is_medecin:
        sql = f"""select o.* from apl.overrepresentation o
        join specialite s on s.psp_spe_snds=o.psp_spe_snds
        where o.year={yy}
        and s.id={specialite}
        """
    else:
        sql = f"""select o.* from apl.overrepresentation o
        join specialite s on s.psp_act_snds=o.psp_act_snds
        where o.year={yy}
        and s.id={specialite}
        """
    over = pd.read_sql(sql, config.connection_string)
    return over

over = get_over(year, specialite, specialite<21)
over

,year,psp_spe_snds,psp_act_snds,p0002,p0305,p0610,p1117,p1824,p2539,p4054,p5564,p6579,p80p
0,23,None,24.0,0.239073,0.22113,0.206107,0.20108,0.375333,0.503999,0.734527,1.16381,2.070894,4.028416


In [32]:
weights = over.values[0,3:]
weights

array([0.23907322106450268, 0.22113007374488441, 0.20610710896092563,
       0.20108049241941214, 0.37533299477824317, 0.5039990020580614,
       0.7345273330087865, 1.163809741174566, 2.0708937918811614,
       4.028415814155608], dtype=object)

In [33]:
apl["pop_gp"] = sum(w * apl[c] for w, c in zip(weights, cols))
# if specialite == 5:
#     apl["pop_gp"] /= 2
apl.sort_values(by='iris2')
apl.head(5)   

,iris1,iris2,km,time_hc,time_hp,time,iris,iris_string,code_commune,type_iris,pop,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4054,pop5564,pop6579,pop80p,accessibilite_weight,pop_gp
0,40080000,60530000,17,22,22,22,60530000,060530000,06053,C,143.0,3.042553,3.042553,3.042553,15.212766,3.042553,19.269504,22.312057,20.283688,39.553191,14.198582,0.071361,195.043475
1,40080000,60990000,21,24,24,24,60990000,060990000,06099,C,1813.0,44.960846,59.174287,115.477201,170.664745,99.908923,286.028889,372.666297,213.784254,309.368287,140.966272,0.056135,1994.686964
2,40390000,61540000,24,30,30,30,61540000,061540000,06154,C,495.0,13.220641,11.542128,38.449393,35.331972,16.699780,71.015373,85.106947,76.715979,113.173907,33.743879,0.027324,584.903633
3,40420000,60530000,11,19,19,19,60530000,060530000,06053,C,143.0,3.042553,3.042553,3.042553,15.212766,3.042553,19.269504,22.312057,20.283688,39.553191,14.198582,0.102284,195.043475
4,40420000,60710000,22,30,32,30,60710000,060710000,06071,C,584.0,12.947819,10.957307,29.872821,24.887047,20.940392,91.782711,104.799327,98.895928,130.928878,57.987769,0.027324,767.610443


In [34]:
apl[apl["iris1"]==61290000]

,iris1,iris2,km,time_hc,time_hp,time,iris,iris_string,code_commune,type_iris,pop,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4054,pop5564,pop6579,pop80p,accessibilite_weight,pop_gp
105136,61290000,60730000,14,17,17,17,60730000,060730000,06073,C,653.0,15.966553,14.947243,42.905088,55.876926,20.971505,96.981393,145.940458,117.149194,107.190565,35.071074,0.130029,690.748860
105137,61290000,60800000,10,18,19,18,60800000,060800000,06080,C,105.0,4.952830,1.981132,3.962264,3.962264,2.971698,10.896226,18.820755,16.839623,35.660377,4.952830,0.115325,137.066015
105138,61290000,60820000,23,30,30,30,60820000,060820000,06082,C,115.0,0.833333,0.833333,2.500000,6.666667,5.833333,13.333333,30.000000,18.333333,25.000000,11.666667,0.027324,153.291598
105139,61290000,61020000,13,21,21,21,61020000,061020000,06102,C,133.0,5.181818,1.727273,4.318182,10.363636,4.318182,12.954545,21.590909,31.954545,29.363636,11.227273,0.080460,171.829772
105140,61290000,61100000,12,19,19,19,61100000,061100000,06110,C,116.0,2.924370,2.924370,2.924370,6.823529,4.873950,11.697479,24.369748,35.092437,16.571429,7.798319,0.102284,135.519196
105141,61290000,61110000,10,17,17,17,61110000,061110000,06111,C,113.0,0.000000,0.000000,1.899160,3.798319,2.848739,15.193277,22.789916,26.588235,25.638655,14.243697,0.130029,168.039753
105142,61290000,61290000,0,0,0,0,61290000,061290000,06129,C,301.0,1.104023,4.449030,15.804142,40.071305,15.409834,45.894488,60.413865,35.273148,57.768530,24.811635,1.000000,346.488197
105143,61290000,61460000,18,23,23,23,61460000,061460000,06146,C,149.0,5.000000,3.000000,11.000000,16.000000,7.000000,23.000000,35.000000,28.000000,13.000000,8.000000,0.063292,139.006606
105144,61290000,61530000,14,21,21,21,61530000,061530000,06153,C,835.0,18.020482,23.026074,44.052137,67.079572,41.031287,120.144366,165.201707,152.189294,156.194759,48.060321,0.080460,923.455127


In [35]:
# apl = apl.drop(["iris", "iris_string", "code_commune"], axis=1)
# apl = apl.drop(cols, axis=1)
apl = apl[["iris1","iris2","km","time","accessibilite_weight","pop_gp","pop","type_iris"]]
apl.head(5)

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris
0,40080000,60530000,17,22,0.071361,195.043475,143.0,C
1,40080000,60990000,21,24,0.056135,1994.686964,1813.0,C
2,40390000,61540000,24,30,0.027324,584.903633,495.0,C
3,40420000,60530000,11,19,0.102284,195.043475,143.0,C
4,40420000,60710000,22,30,0.027324,767.610443,584.0,C


In [36]:
apl["iris"] = apl["iris1"].copy()
apl.sort_values(by='iris2')
apl = apl.merge(df, on="iris", how="left", suffixes=('', ''))
apl

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,code_mode_exercice,key,nb_cabinet,weight,nb
0,40080000,60530000,17,22,0.071361,195.043475,143.000000,C,40080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,40080000,60990000,21,24,0.056135,1994.686964,1813.000000,C,40080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,40390000,61540000,24,30,0.027324,584.903633,495.000000,C,40390000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,40420000,60530000,11,19,0.102284,195.043475,143.000000,C,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,40420000,60710000,22,30,0.027324,767.610443,584.000000,C,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117764,831380000,60850103,34,29,0.030807,3284.455934,2603.604864,A,831380000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117765,831380000,60950101,18,19,0.102284,3707.176334,3223.464405,A,831380000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117766,831380000,60950102,19,22,0.071361,2768.110202,2563.685976,A,831380000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117767,831380000,60950103,18,20,0.090718,2956.445517,2468.849619,A,831380000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
apl["nb"] = apl["nb"].fillna(0)
apl = apl.sort_values(by=['iris1', "iris2"])
apl["nb"].unique()

array([ 0.        ,  4.        ,  2.        ,  1.        ,  5.75      ,
       14.5       ,  1.5       ,  5.5       ,  4.5       , 27.25      ,
       11.        ,  7.        , 26.5       ,  9.25      , 28.        ,
       10.        ,  3.        ,  5.        , 13.5       ,  0.5       ,
        2.5       , 15.        , 12.83333333,  7.5       , 53.5       ,
       13.        ,  4.83333333, 16.        , 60.5       ,  6.5       ,
        6.        ,  1.83333333, 10.5       ,  9.        , 37.        ,
       20.33333333, 12.        , 19.83333333, 12.5       , 25.        ,
        7.33333333,  7.66666667,  8.        ,  9.5       , 13.83333333,
        3.5       , 24.        ,  8.5       ,  1.33333333, 29.5       ,
       16.5       , 21.5       ,  8.83333333, 17.        , 52.        ,
       21.66666667, 24.5       , 10.66666667, 29.        , 28.5       ,
       22.        , 11.5       , 18.        ,  8.33333333, 48.5       ,
        0.33333333, 21.        , 18.5       ,  5.33333333, 26.  

In [38]:
apl = apl.sort_values(by='iris1')
apl["wpop"] = apl["accessibilite_weight"] * apl["pop_gp"]
apl["swpop"] = apl.groupby("iris1")["wpop"].transform("sum")
apl["R"] = apl["nb"] / (apl["swpop"] / 100000)
apl.head(5)
# apl["R"].unique()

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,code_mode_exercice,key,nb_cabinet,weight,nb,wpop,swpop,R
0,40080000,60530000,17,22,0.071361,195.043475,143.0,C,40080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,13.918550,125.889830,0.0
1,40080000,60990000,21,24,0.056135,1994.686964,1813.0,C,40080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,111.971280,125.889830,0.0
2,40390000,61540000,24,30,0.027324,584.903633,495.0,C,40390000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,15.981745,15.981745,0.0
3,40420000,60530000,11,19,0.102284,195.043475,143.0,C,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,19.949867,155.308341,0.0
4,40420000,60710000,22,30,0.027324,767.610443,584.0,C,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,20.973975,155.308341,0.0


In [39]:
rgp = apl[apl["iris1"] == apl["iris2"]]
rgp = rgp[["iris1", "iris2", "type_iris", "pop_gp", "nb", "R", "swpop", "wpop", "pop_gp", "pop"]].copy()
rgp

,iris1,iris2,type_iris,pop_gp,nb,R,swpop,wpop,pop_gp,pop
76,60010000,60010000,C,102.537099,0.00,0.000000,315.583775,102.537099,102.537099,93.000000
87,60020000,60020000,C,67.679719,0.00,0.000000,296.759713,67.679719,67.679719,51.000000
98,60030000,60030000,C,610.798343,0.00,0.000000,1250.898734,610.798343,610.798343,640.000000
112,60040101,60040101,C,1533.346553,4.00,1.998723,200127.827009,1533.346553,1533.346553,1763.364267
477,60040102,60040102,A,2481.307294,2.00,1.249300,160089.603344,2481.307294,2481.307294,2534.265666
...,...,...,...,...,...,...,...,...,...,...
115797,61610105,61610105,A,4292.128570,5.25,2.439037,215248.901627,4292.128570,4292.128570,3584.453834
116147,61610106,61610106,A,4037.463233,6.00,3.834770,156463.103811,4037.463233,4037.463233,3539.143531
116501,61610107,61610107,A,2705.553960,5.00,2.927854,170773.554019,2705.553960,2705.553960,2766.654836
116506,61620000,61620000,C,980.090447,0.00,0.000000,2335.763665,980.090447,980.090447,719.000000


In [40]:
apl = apl.merge(rgp, on="iris2", suffixes=("", "_dest"))
apl.head(5)

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,code_mode_exercice,key,nb_cabinet,weight,nb,wpop,swpop,R,iris1_dest,type_iris_dest,pop_gp_dest,nb_dest,R_dest,swpop_dest,wpop_dest,pop_gp_dest,pop_dest
0,40080000,60530000,17,22,0.071361,195.043475,143.0,C,40080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,13.918550,125.889830,0.0,60530000,C,195.043475,0.0,0.0,448.162398,195.043475,195.043475,143.0
1,40420000,60530000,11,19,0.102284,195.043475,143.0,C,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,19.949867,155.308341,0.0,60530000,C,195.043475,0.0,0.0,448.162398,195.043475,195.043475,143.0
2,40760000,60530000,15,19,0.102284,195.043475,143.0,C,40760000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,19.949867,806.704976,0.0,60530000,C,195.043475,0.0,0.0,448.162398,195.043475,195.043475,143.0
3,40900000,60530000,22,28,0.034735,195.043475,143.0,C,40900000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,6.774886,61.277159,0.0,60530000,C,195.043475,0.0,0.0,448.162398,195.043475,195.043475,143.0
4,41740000,60530000,14,20,0.090718,195.043475,143.0,C,41740000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,17.693945,160.037339,0.0,60530000,C,195.043475,0.0,0.0,448.162398,195.043475,195.043475,143.0


In [41]:
apl = apl.sort_values(by=['iris1', "iris2"])
apl["ap"] = apl["accessibilite_weight"] * apl["R_dest"]
apl["apl"] = apl.groupby("iris1")["ap"].transform("sum")
apl.head(5)

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,code_mode_exercice,key,nb_cabinet,weight,nb,wpop,swpop,R,iris1_dest,type_iris_dest,pop_gp_dest,nb_dest,R_dest,swpop_dest,wpop_dest,pop_gp_dest,pop_dest,ap,apl
0,40080000,60530000,17,22,0.071361,195.043475,143.0,C,40080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,13.918550,125.889830,0.0,60530000,C,195.043475,0.0,0.000000,448.162398,195.043475,195.043475,143.0,0.000000,24.822610
12,40080000,60990000,21,24,0.056135,1994.686964,1813.0,C,40080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,111.971280,125.889830,0.0,60990000,C,1994.686964,10.5,442.196747,2374.508648,1994.686964,1994.686964,1813.0,24.822610,24.822610
33,40390000,61540000,24,30,0.027324,584.903633,495.0,C,40390000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,15.981745,15.981745,0.0,61540000,C,584.903633,5.0,482.213479,1036.885159,584.903633,584.903633,495.0,13.175867,13.175867
1,40420000,60530000,11,19,0.102284,195.043475,143.0,C,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,19.949867,155.308341,0.0,60530000,C,195.043475,0.0,0.000000,448.162398,195.043475,195.043475,143.0,0.000000,32.811238
51,40420000,60710000,22,30,0.027324,767.610443,584.0,C,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,20.973975,155.308341,0.0,60710000,C,767.610443,4.0,292.369693,1368.130861,767.610443,767.610443,584.0,7.988628,32.811238


In [42]:
apl_before=apl
apl = apl[apl["iris1"] == apl["iris2"]]
apl

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,code_mode_exercice,key,nb_cabinet,weight,nb,wpop,swpop,R,iris1_dest,type_iris_dest,pop_gp_dest,nb_dest,R_dest,swpop_dest,wpop_dest,pop_gp_dest,pop_dest,ap,apl
385,60010000,60010000,0,0,1.0,102.537099,93.000000,C,60010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,102.537099,315.583775,0.000000,60010000,C,102.537099,0.00,0.000000,315.583775,102.537099,102.537099,93.000000,0.000000,30.935570
157,60020000,60020000,0,0,1.0,67.679719,51.000000,C,60020000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,67.679719,296.759713,0.000000,60020000,C,67.679719,0.00,0.000000,296.759713,67.679719,67.679719,51.000000,0.000000,0.000000
242,60030000,60030000,0,0,1.0,610.798343,640.000000,C,60030000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,610.798343,1250.898734,0.000000,60030000,C,610.798343,0.00,0.000000,1250.898734,610.798343,610.798343,640.000000,0.000000,84.267523
74748,60040101,60040101,0,0,1.0,1533.346553,1763.364267,C,60040101,95591.0,LESUEUR,AURELIE,6.0,81980.0,7.091032,43.600009,L,95591_43.600009_7.091032,1.0,1.0,4.00,1533.346553,200127.827009,1.998723,60040101,C,1533.346553,4.00,1.998723,200127.827009,1533.346553,1533.346553,1763.364267,1.998723,295.079455
74398,60040102,60040102,0,0,1.0,2481.307294,2534.265666,A,60040102,985120.0,BEN YOUSSEF,HASSINE,6.0,1309164.0,7.068795,43.597024,L,985120_43.597024_7.068795,1.0,1.0,2.00,2481.307294,160089.603344,1.249300,60040102,A,2481.307294,2.00,1.249300,160089.603344,2481.307294,2481.307294,2534.265666,1.249300,237.329916
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39136,61610105,61610105,0,0,1.0,4292.128570,3584.453834,A,61610105,327149.0,GARCIA,CORALIE,6.0,1190804.0,7.133696,43.635700,L,327149_43.6357_7.133696,2.0,0.5,5.25,4292.128570,215248.901627,2.439037,61610105,A,4292.128570,5.25,2.439037,215248.901627,4292.128570,4292.128570,3584.453834,2.439037,314.066174
38772,61610106,61610106,0,0,1.0,4037.463233,3539.143531,A,61610106,1110521.0,VILLAUME,SYLVIE,6.0,444626.0,7.117664,43.635814,L,1110521_43.635814_7.117664,1.0,1.0,6.00,4037.463233,156463.103811,3.834770,61610106,A,4037.463233,6.00,3.834770,156463.103811,4037.463233,4037.463233,3539.143531,3.834770,224.789922
38423,61610107,61610107,0,0,1.0,2705.553960,2766.654836,A,61610107,854519.0,LICARI,STEPHANIE,6.0,398477.0,7.124276,43.655477,L,854519_43.655477_7.124276,1.0,1.0,5.00,2705.553960,170773.554019,2.927854,61610107,A,2705.553960,5.00,2.927854,170773.554019,2705.553960,2705.553960,2766.654836,2.927854,251.614682
116175,61620000,61620000,0,0,1.0,980.090447,719.000000,C,61620000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,980.090447,2335.763665,0.000000,61620000,C,980.090447,0.00,0.000000,2335.763665,980.090447,980.090447,719.000000,0.000000,65.630701


In [44]:
# apl[apl["iris_string"]=="061070000"]

In [45]:
print(year, specialite, source)
apl["apl"].describe()
#for 2021
#spe10 psmean=95 vs 98, psmedian=101 vs 110, after damir changes psmean=95 psmedian=99
#spe10 pa104-109
#spe3 ps7.2-7.8 pa7.8-8.4
#spe7 ps6.4-6.6 pa 7.7-7.7
#spe10 newpa 98-104
#spe3 newpa 7.4-7.8
#spe7 newpa 6.2-6.4
#spe10 pa 99-105 104.5168

23 21 PA


count    500.000000
mean     203.776310
std      108.401653
min        0.000000
25%      125.387039
50%      206.523435
75%      294.691232
max      482.679867
Name: apl, dtype: float64

In [46]:
apl[apl["apl"]>400]

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,code_mode_exercice,key,nb_cabinet,weight,nb,wpop,swpop,R,iris1_dest,type_iris_dest,pop_gp_dest,nb_dest,R_dest,swpop_dest,wpop_dest,pop_gp_dest,pop_dest,ap,apl
27,60990000,60990000,0,0,1.0,1994.686964,1813.0,C,60990000,1025424.0,EUSTACHE,ELODIE,6.0,1249933.0,6.878600,43.954592,L,1025424_43.954592_6.8786,1.0,1.0,10.5,1994.686964,2374.508648,442.196747,60990000,C,1994.686964,10.5,442.196747,2374.508648,1994.686964,1994.686964,1813.0,442.196747,482.679867
116469,61200000,61200000,0,0,1.0,1597.280541,1341.0,C,61200000,836959.0,RIVAS FERRERI,ANNA,6.0,877538.0,6.925037,44.257384,L,836959_44.257384_6.925037,1.0,1.0,8.0,1597.280541,1704.825733,469.256174,61200000,C,1597.280541,8.0,469.256174,1704.825733,1597.280541,1597.280541,1341.0,469.256174,469.256174
43,61540000,61540000,0,0,1.0,584.903633,495.0,C,61540000,1384511.0,GIROLDO,FELIX,6.0,930349.0,6.712699,43.796638,L,1384511_43.796638_6.712699,1.0,1.0,5.0,584.903633,1036.885159,482.213479,61540000,C,584.903633,5.0,482.213479,1036.885159,584.903633,584.903633,495.0,482.213479,482.213479


In [47]:
apl = apl[["iris1", "type_iris", "nb", "apl", "ap", "accessibilite_weight", "wpop", "swpop", "R", "pop_gp","pop"]]
apl = apl.rename(columns={'iris1': 'iris'})
apl.head(5)

,iris,type_iris,nb,apl,ap,accessibilite_weight,wpop,swpop,R,pop_gp,pop
385,60010000,C,0.0,30.935570,0.000000,1.0,102.537099,315.583775,0.000000,102.537099,93.000000
157,60020000,C,0.0,0.000000,0.000000,1.0,67.679719,296.759713,0.000000,67.679719,51.000000
242,60030000,C,0.0,84.267523,0.000000,1.0,610.798343,1250.898734,0.000000,610.798343,640.000000
74748,60040101,C,4.0,295.079455,1.998723,1.0,1533.346553,200127.827009,1.998723,1533.346553,1763.364267
74398,60040102,A,2.0,237.329916,1.249300,1.0,2481.307294,160089.603344,1.249300,2481.307294,2534.265666


In [48]:
def get_iriss():
    sql = f"""
select i.id "iris", i.code "iris_string", i.nom "iris_label", c.dept_id "dept", c.code "code_commune", c.nom "commune_label", 20{year} "year", {specialite} "specialite" from iris.iris i
join iris.commune c on c.id=i.commune_id
where i.code like '{dept:02d}%'"""
    print(sql)
    return pd.read_sql(text(sql), config.connection_string)

iriss = get_iriss()
iriss


select i.id "iris", i.code "iris_string", i.nom "iris_label", c.dept_id "dept", c.code "code_commune", c.nom "commune_label", 2023 "year", 21 "specialite" from iris.iris i
join iris.commune c on c.id=i.commune_id
where i.code like '06%'


,iris,iris_string,iris_label,dept,code_commune,commune_label,year,specialite
0,60120105,060120105,Centre,6,06012,Beausoleil,2023,21
1,60500000,060500000,Coursegoules,6,06050,Coursegoules,2023,21
2,60720000,060720000,Ilonse,6,06072,Ilonse,2023,21
3,61050101,061050101,Nord,6,06105,Roquefort-les-Pins,2023,21
4,60150000,060150000,Berre-les-Alpes,6,06015,Berre-les-Alpes,2023,21
...,...,...,...,...,...,...,...,...
495,60883403,060883403,Madeleine Supérieure,6,06088,Nice,2023,21
496,60883501,060883501,Crémat-Bellet,6,06088,Nice,2023,21
497,60883601,060883601,Saint-Pierre de Féric,6,06088,Nice,2023,21
498,60883602,060883602,Costière,6,06088,Nice,2023,21


In [49]:
apl = apl.merge(iriss, on="iris", how="left", suffixes=("", "_dest"))
apl = apl[["year", "specialite", "iris", "iris_string", "type_iris", "nb", "apl", "ap", "accessibilite_weight", "R", "wpop", "swpop", "pop_gp","pop", "iris_label", "dept", "code_commune", "commune_label"]]
apl

,year,specialite,iris,iris_string,type_iris,nb,apl,ap,accessibilite_weight,R,wpop,swpop,pop_gp,pop,iris_label,dept,code_commune,commune_label
0,2023,21,60010000,060010000,C,0.00,30.935570,0.000000,1.0,0.000000,102.537099,315.583775,102.537099,93.000000,Aiglun,6,06001,Aiglun
1,2023,21,60020000,060020000,C,0.00,0.000000,0.000000,1.0,0.000000,67.679719,296.759713,67.679719,51.000000,Amirat,6,06002,Amirat
2,2023,21,60030000,060030000,C,0.00,84.267523,0.000000,1.0,0.000000,610.798343,1250.898734,610.798343,640.000000,Andon,6,06003,Andon
3,2023,21,60040101,060040101,C,4.00,295.079455,1.998723,1.0,1.998723,1533.346553,200127.827009,1533.346553,1763.364267,Les Trois Moulins,6,06004,Antibes
4,2023,21,60040102,060040102,A,2.00,237.329916,1.249300,1.0,1.249300,2481.307294,160089.603344,2481.307294,2534.265666,Les Semboules,6,06004,Antibes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2023,21,61610105,061610105,A,5.25,314.066174,2.439037,1.0,2.439037,4292.128570,215248.901627,4292.128570,3584.453834,Marina,6,06161,Villeneuve-Loubet
496,2023,21,61610106,061610106,A,6.00,224.789922,3.834770,1.0,3.834770,4037.463233,156463.103811,4037.463233,3539.143531,Vaugrenier,6,06161,Villeneuve-Loubet
497,2023,21,61610107,061610107,A,5.00,251.614682,2.927854,1.0,2.927854,2705.553960,170773.554019,2705.553960,2766.654836,Plan,6,06161,Villeneuve-Loubet
498,2023,21,61620000,061620000,C,0.00,65.630701,0.000000,1.0,0.000000,980.090447,2335.763665,980.090447,719.000000,La Brigue,6,06162,La Brigue


In [50]:
# Anomalie = 1 seul medecin avec 1 seul cabinet pour 300 habitants contre 1 seul medecin et 0.33 cabinets
# La commune de sauveur en tinee a un APL > 400 pour les généralistes car l'iris a seulement 300 habitants jeunes et ils sont passés de 1 médecin avec 3 cabinets à 1 médecin a un seul cabinet en 2022
apl[apl["iris_string"]=="061070000"]


,year,specialite,iris,iris_string,type_iris,nb,apl,ap,accessibilite_weight,R,wpop,swpop,pop_gp,pop,iris_label,dept,code_commune,commune_label
404,2023,21,61070000,061070000,C,0.0,149.836336,0.0,1.0,0.0,60.81118,913.958236,60.81118,65.0,La Roque-en-Provence,6,06107,La Roque-en-Provence


In [51]:
# apl.to_csv("apl06.csv", index=False)